# Memgraph as Graph Memory

## Prerequisites

### 1. Install Mem0 with Graph Memory support 

To use Mem0 with Graph Memory support, install it using pip:

```bash
pip install "mem0ai[graph]"
```

This command installs Mem0 along with the necessary dependencies for graph functionality.

### 2. Install Memgraph

To utilize Memgraph as Graph Memory, run it with Docker:

```bash
docker run -p 7687:7687 memgraph/memgraph-mage:latest --schema-info-enabled=True
```

The `--schema-info-enabled` flag is set to `True` for more performant schema
generation.

Additional information can be found on [Memgraph documentation](https://memgraph.com/docs). 

## Configuration

Do all the imports and configure OpenAI (enter your OpenAI API key):

In [1]:
from mem0 import Memory

import os


os.environ["OPENAI_API_KEY"] = ""
os.environ["LLM_AZURE_OPENAI_API_KEY"] = "FjfmeNsmd6aBBbCOWLb4sl8RU0057djGvmGcvzhqYrOkUtifGvd0JQQJ99BEACHYHv6XJ3w3AAAAACOGIlEZ"
os.environ["LLM_AZURE_DEPLOYMENT"] = "gpt-4o-mini"
os.environ["LLM_AZURE_ENDPOINT"] = "https://123s-mann562s-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"
os.environ["LLM_AZURE_API_VERSION"] = "2025-01-01-preview"


Set up configuration to use the embedder model and Memgraph as a graph store:

In [2]:
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4o-mini",
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                  "azure_deployment": "gpt-4o-mini",
                  "api_version": "2025-01-01-preview",
                  "azure_endpoint": "https://123s-mann562s-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview",
                  "api_key": "FjfmeNsmd6aBBbCOWLb4sl8RU0057djGvmGcvzhqYrOkUtifGvd0JQQJ99BEACHYHv6XJ3w3AAAAACOGIlEZ",
              }
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-3-small",
            "azure_kwargs": {
                "azure_deployment": "text-embedding-3-small",
                "api_version": "2023-05-15",
                "azure_endpoint": "https://123s-mann562s-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15",
                "api_key": "FjfmeNsmd6aBBbCOWLb4sl8RU0057djGvmGcvzhqYrOkUtifGvd0JQQJ99BEACHYHv6XJ3w3AAAAACOGIlEZ",
            },
            "embedding_dims": 1536,
        }
    },
    "graph_store": {
        "provider": "memgraph",
        "config": {
            "url": "bolt://localhost:7687",
            "username": "memgraph",
            "password": "mem0graph",
        },
    },
}

## Graph Memory initializiation 

Initialize Memgraph as a Graph Memory store: 

In [4]:
m = Memory.from_config(config_dict=config)

## Store memories 

Create memories:

In [5]:
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]

Store memories in Memgraph:

In [6]:
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})

![](./alice-memories.png)

## Search memories

In [7]:
for result in m.search("what does alice love?", user_id="alice")["results"]:
    print(result["memory"], result["score"])

Loves sci-fi movies 0.2619825285692935
Planning to watch a movie tonight 0.06464504330066467
Not a big fan of thriller movies 0.054711775840949814
